In [1]:
import networkx as nx
import numpy as np
from hiveplot import HivePlot
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
pd_data = pd.read_table('kg_without_inconsistencies.txt', sep='\t')
pd_data.head()

,Subject,Predicate,Object,Belief,Source size,Sources
0,AAC(3)-Ia,confers resistance to antibiotic,Aminoglycosides,0.28,1,Salm
1,AAC(3)-Ia,confers resistance to antibiotic,Gentamicin,0.28,1,Salm
2,AAC(3)-Ia,confers resistance to antibiotic,Streptomycin,0.28,1,Salm
3,AAC(3)-Ia,has,response to antibiotic,0.28,1,Salm
4,AAC(3)-VIa,confers resistance to antibiotic,Aminoglycosides,0.28,1,Salm


In [8]:
neg_predicates = [
    'confers no resistance to antibiotic',
    'is not regulated by antibiotic',
    'is not mutated by antibiotic',
    'no represses',
    'no activates',]

def _check_match(x, predicates):
    flag = False
    for predicate in predicates:
        if predicate in x:
            flag = True
    return flag



# remove the negatives
pd_data = pd_data[~pd_data['Predicate'].apply(lambda x: _check_match(x, neg_predicates))]

# select data by relation type
pd_cras = pd_data[pd_data['Predicate'].str.contains('resistance to antibiotic')]
pd_ubas = pd_data[pd_data['Predicate'].str.contains('is upregulated by antibiotic')]
pd_represses = pd_data[pd_data['Predicate'].str.contains('represses')]
pd_activates = pd_data[pd_data['Predicate'].str.contains('activates')]
pd_has = pd_data[pd_data['Predicate'].str.contains('has')]
pd_iii = pd_data[pd_data['Predicate'].str.contains('is involved in')]
pd_ipo = pd_data[pd_data['Predicate'].str.contains('is part of')]
pd_tb = pd_data[pd_data['Predicate'].str.contains('targeted by')]
pd_imb = pd_data[pd_data['Predicate'].str.contains('is mutated by antibiotic')]
pd_inmb = pd_data[pd_data['Predicate'].str.contains('is not mutated by antibiotic')]
pd_iaw = pd_data[pd_data['Predicate'].str.contains('is associated with')]
pd_idba = pd_data[pd_data['Predicate'].str.contains('is downregulated by antibiotic')]

# get genes
genes = []
genes.extend(pd_cras['Subject'].to_numpy().tolist())
genes.extend(pd_ubas['Subject'].to_numpy().tolist())
genes.extend(pd_represses['Subject'].to_numpy().tolist())
genes.extend(pd_represses['Object'].to_numpy().tolist())
genes.extend(pd_activates['Subject'].to_numpy().tolist())
genes.extend(pd_activates['Object'].to_numpy().tolist())
genes.extend(pd_has['Subject'].to_numpy().tolist())
genes.extend(pd_iii['Subject'].to_numpy().tolist())
genes.extend(pd_ipo['Subject'].to_numpy().tolist())
genes.extend(pd_tb['Subject'].to_numpy().tolist())
genes.extend(pd_imb['Subject'].to_numpy().tolist())
genes.extend(pd_inmb['Subject'].to_numpy().tolist())
genes.extend(pd_iaw['Subject'].to_numpy().tolist())
genes.extend(pd_idba['Subject'].to_numpy().tolist())

genes = list(set(genes))

pd_genes = pd.DataFrame(genes, columns=['Label'])
pd_genes['Category'] = 'gene'
print('gene:', pd_genes.shape)

# get antibiotics
antibiotics = []
antibiotics.extend(pd_cras['Object'].to_numpy().tolist())
antibiotics.extend(pd_ubas['Object'].to_numpy().tolist())
antibiotics.extend(pd_tb['Object'].to_numpy().tolist())
antibiotics = list(set(antibiotics))

pd_antibiotics = pd.DataFrame(antibiotics, columns=['Label'])
pd_antibiotics['Category'] = 'antibiotic'
print('antibiotic:', pd_antibiotics.shape)

gene: (4335, 2)
antibiotic: (81, 2)


In [9]:
# get molecular_function
molecular_functions = pd_has['Object'].to_numpy().tolist()
molecular_functions = list(set(molecular_functions))

pd_molecular_functions = pd.DataFrame(molecular_functions, columns=['Label'])
pd_molecular_functions['Category'] = 'molecular_function'
print('molecular_function:', pd_molecular_functions.shape)

# get biological_process
biological_processes = pd_iii['Object'].to_numpy().tolist()
biological_processes = list(set(biological_processes))

pd_biological_processes = pd.DataFrame(biological_processes, columns=['Label'])
pd_biological_processes['Category'] = 'biological_process'
print('biological_process:', pd_biological_processes.shape)

# get cellular_component
cellular_components = pd_ipo['Object'].to_numpy().tolist()
cellular_components = list(set(cellular_components))

pd_cellular_components = pd.DataFrame(cellular_components, columns=['Label'])
pd_cellular_components['Category'] = 'cellular_component'
print('cellular_component:', pd_cellular_components.shape)

molecular_function: (2031, 2)
biological_process: (2055, 2)
cellular_component: (173, 2)


In [12]:
pd_nodes = pd.concat(
    [pd_genes,
    pd_antibiotics,
    pd_molecular_functions,
    pd_biological_processes,
    pd_cellular_components])

pd_nodes = pd_nodes.reset_index(drop=True)
pd_nodes.head()

,Label,Category
0,hypC,gene
1,MCR-4.1,gene
2,STM14_1307,gene
3,STM14_1952,gene
4,tolA,gene


In [15]:
pd_edges = pd_data.copy()

def map_func(label):
    if label.startswith('confers resistance to antibiotic'):
        return 'confers resistance to antibiotic'
    elif label.startswith('confers no resistance to antibiotic'):
        return 'confers no resistance to antibiotic'
    elif label.startswith('is upregulated by antibiotic'):
        return 'is  upregulated by antibiotic'
    elif label.startswith('is not regulated by antibiotic'):
        return 'is not regulated by antibiotic'
    elif 'represses' == label:
        return 'represses'
    elif 'no represses' == label:
        return 'no represses'
    elif 'activates ' == label:
        return 'activates'
    elif 'no activates' == label:
        return 'no activates'
    elif 'has' == label:
        return 'has'
    elif 'is involved in' == label:
        return 'is involved in'
    elif 'is part of' == label:
        return 'is part of'
    elif 'targeted by' == label:
        return 'targeted by'
    elif 'is mutated by antibiotic' == label:
        return 'is mutated by antibiotic'
    elif 'is associated with resistance to antibiotic' ==  label:
        return 'is associated with resistance to antibiotic'
    elif 'is downregulated by antibiotic' == label:
        return 'is downregulated by antibiotic'
    elif 'is not mutated by antibiotic' == label:
        return 'is not mutated by antibiotic'   
    
    else:
        raise ValueError('Invalid label: {}'.format(label))

pd_edges['Category'] = pd_edges['Predicate'].apply(lambda x: map_func(x))
pd_edges.head()

,Subject,Predicate,Object,Belief,Source size,Sources,Category
0,AAC(3)-Ia,confers resistance to antibiotic,Aminoglycosides,0.28,1,Salm,confers resistance to antibiotic
1,AAC(3)-Ia,confers resistance to antibiotic,Gentamicin,0.28,1,Salm,confers resistance to antibiotic
2,AAC(3)-Ia,confers resistance to antibiotic,Streptomycin,0.28,1,Salm,confers resistance to antibiotic
3,AAC(3)-Ia,has,response to antibiotic,0.28,1,Salm,has
4,AAC(3)-VIa,confers resistance to antibiotic,Aminoglycosides,0.28,1,Salm,confers resistance to antibiotic


In [16]:
# build graph
G = nx.MultiDiGraph()

for _, row in pd_nodes.iterrows():
    G.add_node(row['Label'], category=row['Category'])

for _, row in pd_edges.iterrows():
    G.add_edge(row['Subject'], row['Object'], category=row['Category'])


In [17]:

for n, d in G.nodes(data=True):
	print(n)
	print(d)
	break

for u, v, c in G.edges.data():
	print(u)
	print(v)
	print(c)
	break

hypC
{'category': 'gene'}
hypC
binding
{'category': 'has'}


In [19]:
# hive plot
nodes = dict()
# nodes['gene'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'gene']
# nodes['antibiotic'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'antibiotic']
# nodes['molecular_function'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'molecular_function']
# nodes['biological_process'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'biological_process']
# nodes['cellular_component'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'cellular_component']

nodes['gene'] = [n for n, d in G.nodes(data=True) if d['category'] == 'gene']
nodes['antibiotic'] = [n for n, d in G.nodes(data=True) if d['category'] == 'antibiotic']
nodes['molecular_function'] = [n for n, d in G.nodes(data=True) if d['category'] == 'molecular_function']
nodes['biological_process'] = [n for n, d in G.nodes(data=True) if d['category'] == 'biological_process']
nodes['cellular_component'] = [n for n, d in G.nodes(data=True) if d['category'] == 'cellular_component']

edges = dict()
edges['cra'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'confers resistance to antibiotic']
edges['uba'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'upregulated by antibiotic']
edges['represses'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'represses']
edges['activates'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'activates']
edges['has'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'has']
edges['iii'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is involved in']
edges['ipo'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is part of']
edges['tb'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'targeted by']
edges['inmb'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is not mutated by antibiotic']
edges['imb'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is mutated by antibiotic']
edges['isw'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is associated with resistance to antibiotic']
edges['idba'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is downregulated by antibiotic']

edges

{'cra': [('MCR-4.1',
   'Colistin',
   {'category': 'confers resistance to antibiotic'}),
  ('t0533', 'Ciprofloxacin', {'category': 'confers resistance to antibiotic'}),
  ('fosL', 'Fosfomycin', {'category': 'confers resistance to antibiotic'}),
  ('qnrB1', 'Quinolone', {'category': 'confers resistance to antibiotic'}),
  ('blaSCO', 'Beta-Lactam', {'category': 'confers resistance to antibiotic'}),
  ('aph4-Ia',
   'Aminoglycosides',
   {'category': 'confers resistance to antibiotic'}),
  ('aph4-Ia', 'Gentamicin', {'category': 'confers resistance to antibiotic'}),
  ('aph4-Ia',
   'Streptomycin',
   {'category': 'confers resistance to antibiotic'}),
  ('qnrVC4', 'Quinolone', {'category': 'confers resistance to antibiotic'}),
  ('acrZ', 'Ciprofloxacin', {'category': 'confers resistance to antibiotic'}),
  ('aacA38',
   'Aminoglycoside',
   {'category': 'confers resistance to antibiotic'}),
  ("aac(6')-Ib-cr4",
   'Amikacin',
   {'category': 'confers resistance to antibiotic'}),
  ("aac(6

In [21]:

nodes_cmap = dict()
nodes_cmap['gene'] = 'red'
nodes_cmap['antibiotic'] = 'green'
nodes_cmap['molecular_function'] = 'blue'
nodes_cmap['biological_process'] = 'pink'
nodes_cmap['cellular_component'] = 'magenta'

edges_cmap = dict()
edges_cmap['cra'] = 'gray'
edges_cmap['uba'] = 'gray'
edges_cmap['represses'] = 'gray'
edges_cmap['activates'] = 'gray'
edges_cmap['has'] = 'gray'
edges_cmap['iii'] = 'gray'
edges_cmap['ipo'] = 'gray'
edges_cmap['tb'] = 'gray'
edges_cmap['imb'] = 'gray'
edges_cmap['inmb'] = 'gray'
edges_cmap['iaw'] = 'gray'
edges_cmap['idba'] = 'gray'
nodes_cmap

{'gene': 'red',
 'antibiotic': 'green',
 'molecular_function': 'blue',
 'biological_process': 'pink',
 'cellular_component': 'magenta'}

In [ ]:
h = HivePlot(nodes, edges, nodes_cmap, edges_cmap, linewidth=0.1, is_directed=True, scale=30)
h.draw()
plt.show()

In [ ]:
import networkx as nx
import numpy as np
from hiveplot import HivePlot
import pandas as pd
import matplotlib.pyplot as plt

# load data
pd_data = pd.read_table('kg_without_inconsistencies.txt', sep='\t')
pd_data = pd_data[['Subject', 'Predicate', 'Object']]

# filter positives only
neg_predicates = [
    'confers no resistance to antibiotic',
    'is not regulated by antibiotic',
    'is not mutated by antibiotic',
    'no represses',
    'no activates',]

def _check_match(x, predicates):
    flag = False
    for predicate in predicates:
        if predicate in x:
            flag = True
    return flag

# remove the negatives
pd_data = pd_data[~pd_data['Predicate'].apply(lambda x: _check_match(x, neg_predicates))]

# select data by relation type
pd_cras = pd_data[pd_data['Predicate'].str.contains('resistance to antibiotic')]
pd_ubas = pd_data[pd_data['Predicate'].str.contains('is upregulated by antibiotic')]
pd_represses = pd_data[pd_data['Predicate'].str.contains('represses')]
pd_activates = pd_data[pd_data['Predicate'].str.contains('activates')]
pd_has = pd_data[pd_data['Predicate'].str.contains('has')]
pd_iii = pd_data[pd_data['Predicate'].str.contains('is involved in')]
pd_ipo = pd_data[pd_data['Predicate'].str.contains('is part of')]
pd_tb = pd_data[pd_data['Predicate'].str.contains('targeted by')]
pd_imb = pd_data[pd_data['Predicate'].str.contains('is mutated by antibiotic')]
pd_inmb = pd_data[pd_data['Predicate'].str.contains('is not mutated by antibiotic')]
pd_iaw = pd_data[pd_data['Predicate'].str.contains('is associated with')]
pd_idba = pd_data[pd_data['Predicate'].str.contains('is downregulated by antibiotic')]

# get genes
genes = []
genes.extend(pd_cras['Subject'].to_numpy().tolist())
genes.extend(pd_ubas['Subject'].to_numpy().tolist())
genes.extend(pd_represses['Subject'].to_numpy().tolist())
genes.extend(pd_represses['Object'].to_numpy().tolist())
genes.extend(pd_activates['Subject'].to_numpy().tolist())
genes.extend(pd_activates['Object'].to_numpy().tolist())
genes.extend(pd_has['Subject'].to_numpy().tolist())
genes.extend(pd_iii['Subject'].to_numpy().tolist())
genes.extend(pd_ipo['Subject'].to_numpy().tolist())
genes.extend(pd_tb['Subject'].to_numpy().tolist())
genes.extend(pd_imb['Subject'].to_numpy().tolist())
genes.extend(pd_inmb['Subject'].to_numpy().tolist())
genes.extend(pd_iaw['Subject'].to_numpy().tolist())
genes.extend(pd_idba['Subject'].to_numpy().tolist())

genes = list(set(genes))

pd_genes = pd.DataFrame(genes, columns=['Label'])
pd_genes['Category'] = 'gene'
print('gene:', pd_genes.shape)

# get antibiotics
antibiotics = []
antibiotics.extend(pd_cras['Object'].to_numpy().tolist())
antibiotics.extend(pd_ubas['Object'].to_numpy().tolist())
antibiotics.extend(pd_tb['Object'].to_numpy().tolist())
antibiotics = list(set(antibiotics))

pd_antibiotics = pd.DataFrame(antibiotics, columns=['Label'])
pd_antibiotics['Category'] = 'antibiotic'
print('antibiotic:', pd_antibiotics.shape)

# get molecular_function
molecular_functions = pd_has['Object'].to_numpy().tolist()
molecular_functions = list(set(molecular_functions))

pd_molecular_functions = pd.DataFrame(molecular_functions, columns=['Label'])
pd_molecular_functions['Category'] = 'molecular_function'
print('molecular_function:', pd_molecular_functions.shape)

# get biological_process
biological_processes = pd_iii['Object'].to_numpy().tolist()
biological_processes = list(set(biological_processes))

pd_biological_processes = pd.DataFrame(biological_processes, columns=['Label'])
pd_biological_processes['Category'] = 'biological_process'
print('biological_process:', pd_biological_processes.shape)

# get cellular_component
cellular_components = pd_ipo['Object'].to_numpy().tolist()
cellular_components = list(set(cellular_components))

pd_cellular_components = pd.DataFrame(cellular_components, columns=['Label'])
pd_cellular_components['Category'] = 'cellular_component'
print('cellular_component:', pd_cellular_components.shape)

#########
# nodes #
#########
pd_nodes = pd.concat(
    [pd_genes,
    pd_antibiotics,
    pd_molecular_functions,
    pd_biological_processes,
    pd_cellular_components])

pd_nodes = pd_nodes.reset_index(drop=True)

#########
# edges #
#########
pd_edges = pd_data.copy()

def map_func(label):
    if label.startswith('confers resistance to antibiotic'):
        return 'confers resistance to antibiotic'
    elif label.startswith('confers no resistance to antibiotic'):
        return 'confers no resistance to antibiotic'
    elif label.startswith('is upregulated by antibiotic'):
        return 'is  upregulated by antibiotic'
    elif label.startswith('is not regulated by antibiotic'):
        return 'is not regulated by antibiotic'
    elif 'represses' == label:
        return 'represses'
    elif 'no represses' == label:
        return 'no represses'
    elif 'activates ' == label:
        return 'activates'
    elif 'no activates' == label:
        return 'no activates'
    elif 'has' == label:
        return 'has'
    elif 'is involved in' == label:
        return 'is involved in'
    elif 'is part of' == label:
        return 'is part of'
    elif 'targeted by' == label:
        return 'targeted by'
    elif 'is mutated by antibiotic' == label:
        return 'is mutated by antibiotic'
    elif 'is associated with resistance to antibiotic' ==  label:
        return 'is associated with resistance to antibiotic'
    elif 'is downregulated by antibiotic' == label:
        return 'is downregulated by antibiotic'
    elif 'is not mutated by antibiotic' == label:
        return 'is not mutated by antibiotic'   
    
    else:
        raise ValueError('Invalid label: {}'.format(label))

pd_edges['Category'] = pd_edges['Predicate'].apply(lambda x: map_func(x))

# build graph
G = nx.MultiDiGraph()

for _, row in pd_nodes.iterrows():
    G.add_node(row['Label'], category=row['Category'])

for _, row in pd_edges.iterrows():
    G.add_edge(row['Subject'], row['Object'], category=row['Category'])

for n, d in G.nodes(data=True):
	print(n)
	print(d)
	break

for u, v, c in G.edges.data():
	print(u)
	print(v)
	print(c)
	break

# hive plot
nodes = dict()
# nodes['gene'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'gene']
# nodes['antibiotic'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'antibiotic']
# nodes['molecular_function'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'molecular_function']
# nodes['biological_process'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'biological_process']
# nodes['cellular_component'] = [(n,d) for n, d in G.nodes(data=True) if d['category'] == 'cellular_component']

nodes['gene'] = [n for n, d in G.nodes(data=True) if d['category'] == 'gene']
nodes['antibiotic'] = [n for n, d in G.nodes(data=True) if d['category'] == 'antibiotic']
nodes['molecular_function'] = [n for n, d in G.nodes(data=True) if d['category'] == 'molecular_function']
nodes['biological_process'] = [n for n, d in G.nodes(data=True) if d['category'] == 'biological_process']
nodes['cellular_component'] = [n for n, d in G.nodes(data=True) if d['category'] == 'cellular_component']

edges = dict()
edges['cra'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'confers resistance to antibiotic']
edges['uba'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'upregulated by antibiotic']
edges['represses'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'represses']
edges['activates'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'activates']
edges['has'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'has']
edges['iii'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is involved in']
edges['ipo'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is part of']
edges['tb'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'targeted by']
edges['inmb'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is not mutated by antibiotic']
edges['imb'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is mutated by antibiotic']
edges['isw'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is associated with resistance to antibiotic']
edges['idba'] = [(u,v,d) for u,v,d in G.edges.data() if d['category'] == 'is downregulated by antibiotic']



nodes_cmap = dict()
nodes_cmap['gene'] = 'red'
nodes_cmap['antibiotic'] = 'green'
nodes_cmap['molecular_function'] = 'blue'
nodes_cmap['biological_process'] = 'pink'
nodes_cmap['cellular_component'] = 'magenta'

edges_cmap = dict()
edges_cmap['cra'] = 'gray'
edges_cmap['uba'] = 'gray'
edges_cmap['represses'] = 'gray'
edges_cmap['activates'] = 'gray'
edges_cmap['has'] = 'gray'
edges_cmap['iii'] = 'gray'
edges_cmap['ipo'] = 'gray'
edges_cmap['tb'] = 'gray'
edges_cmap['imb'] = 'gray'
edges_cmap['inmb'] = 'gray'
edges_cmap['iaw'] = 'gray'
edges_cmap['idba'] = 'gray'

h = HivePlot(nodes, edges, nodes_cmap, edges_cmap, linewidth=0.1, is_directed=True, scale=30)
h.draw()
plt.show()